In [2]:
import pandas as pd
import sqlite3
from tqdm import tqdm
import os

conn = sqlite3.connect('mimic3.db')

In [3]:
# print column names
print(pd.read_sql_query("SELECT * FROM patients LIMIT 1", conn))

# load chartevents table and get the first 5 rows
df = pd.read_sql_query("SELECT * FROM chartevents LIMIT 5", conn)
print(df)

   ROW_ID  SUBJECT_ID GENDER                  DOB   DOD DOD_HOSP DOD_SSN  \
0     234         249      F  2075-03-13 00:00:00  None     None    None   

   EXPIRE_FLAG  
0            0  
   ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID  ITEMID            CHARTTIME  \
0     788          36   165660    241249.0  223834  2134-05-12 12:00:00   
1     789          36   165660    241249.0  223835  2134-05-12 12:00:00   
2     790          36   165660    241249.0  224328  2134-05-12 12:00:00   
3     791          36   165660    241249.0  224329  2134-05-12 12:00:00   
4     792          36   165660    241249.0  224330  2134-05-12 12:00:00   

             STORETIME     CGID   VALUE  VALUENUM VALUEUOM  WARNING  ERROR  \
0  2134-05-12 13:56:00  17525.0   15.00     15.00    L/min        0      0   
1  2134-05-12 13:56:00  17525.0  100.00    100.00     None        0      0   
2  2134-05-12 12:18:00  20823.0    0.37      0.37     None        0      0   
3  2134-05-12 12:19:00  20823.0    6.00      6.00 

In [7]:
GCS_ITEM_IDs = []
Ve_ITEM_IDs = []
Vt_ITEM_IDs = []
MBP_ITEM_IDs = []
HR_ITEM_IDs = []
RR_ITEM_IDs = []
PiMax_ITEM_IDs = []
PaCO2_ITEM_IDs = []

# open d_items and get the itemid that includes GCS in the label : GCS
df = pd.read_sql_query("SELECT * FROM d_items WHERE label LIKE '%GCS%'", conn)
print(df)
for i in range(len(df)):
    GCS_ITEM_IDs.append(df['ITEMID'][i])
# get only 198, 227013, 226755
GCS_ITEM_IDs = [198, 227013, 226755]

# open d_items and get the itemid that includes "minute Ve" in the label, find considering lower cases : Ve
df = pd.read_sql_query("SELECT * FROM d_items WHERE label LIKE '%Minute Ve%'", conn)
for i in range(len(df)):
    Ve_ITEM_IDs.append(df['ITEMID'][i])
print("Ve itemid: ", Ve_ITEM_IDs)

# open d_items and get the itemid that includes "minute Vt" in the label : Vt
df = pd.read_sql_query("SELECT * FROM d_items WHERE label LIKE '%Vt%'", conn)
for i in range(len(df)):
    Vt_ITEM_IDs.append(df['ITEMID'][i])
print("Vt itemid: ", Vt_ITEM_IDs)

# open d_items and get the itemid that includes "MAP" in the label : MBP
df = pd.read_sql_query("SELECT * FROM d_items WHERE label LIKE '%MAP%'", conn)
for i in range(len(df)):
    MBP_ITEM_IDs.append(df['ITEMID'][i])
print("MBP itemid: ", MBP_ITEM_IDs)

# open d_items and get the itemid that includes "HR" in the label : HR
df = pd.read_sql_query("SELECT * FROM d_items WHERE label LIKE '%heart rate%'", conn)
for i in range(len(df)):
    HR_ITEM_IDs.append(df['ITEMID'][i])
print("HR itemid: ", HR_ITEM_IDs)

# open d_items and get the itemid that includes "RR" in the label : RR
df = pd.read_sql_query("SELECT * FROM d_items WHERE label LIKE '%respiratory rate%'", conn)
for i in range(len(df)):
    RR_ITEM_IDs.append(df['ITEMID'][i])
print("RR itemid: ", RR_ITEM_IDs)

# open d_items and get the itemid that includes "PiMax" in the label : PiMax
df = pd.read_sql_query("SELECT * FROM d_items WHERE label LIKE '%inspiratory%'", conn)
for i in range(len(df)):
    PiMax_ITEM_IDs.append(df['ITEMID'][i])
print("PiMax itemid: ", PiMax_ITEM_IDs)

# open d_items and get the itemid that includes "PaCO2" in the label : PaCO2
df = pd.read_sql_query("SELECT * FROM d_items WHERE label LIKE '%PaCO2%'", conn)
for i in range(len(df)):
    PaCO2_ITEM_IDs.append(df['ITEMID'][i])
print("PaCO2 itemid: ", PaCO2_ITEM_IDs)



    ROW_ID  ITEMID                               LABEL  \
0      199     198                           GCS Total   
1    14688  227011                     GCSEye_ApacheIV   
2    14689  227012                   GCSMotor_ApacheIV   
3    14690  227013                   GcsScore_ApacheIV   
4    14691  227014                  GCSVerbal_ApacheIV   
5    12708  220739                   GCS - Eye Opening   
6    15305  228112  GCSVerbalApacheIIValue (intubated)   
7    12827  223900               GCS - Verbal Response   
8    12828  223901                GCS - Motor Response   
9    14516  226755                    GcsApacheIIScore   
10   14517  226756                 GCSEyeApacheIIValue   
11   14518  226757               GCSMotorApacheIIValue   
12   14519  226758              GCSVerbalApacheIIValue   

                          ABBREVIATION    DBSOURCE      LINKSTO  \
0                                 None     carevue  chartevents   
1                      GCSEye_ApacheIV  metavision  c

In [8]:
# load chartevents table and find the rows with GCS_ITEM_IDs, get only 100 rows
df = pd.read_sql_query("SELECT * FROM chartevents WHERE itemid IN ({}) LIMIT 10".format(','.join(['?']*len(GCS_ITEM_IDs))), conn, params=GCS_ITEM_IDs)
print(df)


    ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID  ITEMID            CHARTTIME  \
0    85994         711   158767    270525.0  226755  2185-03-23 14:26:00   
1    86034         711   158767    270525.0  227013  2185-03-23 14:29:00   
2    86076         711   158767    270525.0  227013  2185-03-23 14:34:00   
3    86119         711   158767    270525.0  226755  2185-03-23 15:07:00   
4  1150560       10757   153982    256332.0  226755  2180-11-14 00:26:00   
5  2658460       22792   192087    271490.0  226755  2173-10-23 07:07:00   
6  3215052       27121   167456    211255.0  227013  2126-12-09 14:32:00   
7  3202256       27109   103103    237073.0  227013  2195-05-20 13:08:00   
8  3301308       27366   174736    248077.0  226755  2143-03-01 09:16:00   
9  3301382       27366   174736    248077.0  226755  2143-03-01 11:24:00   

             STORETIME     CGID  VALUE  VALUENUM VALUEUOM  WARNING  ERROR  \
0  2185-03-23 14:29:00  19006.0    7.0       7.0     None        0      0   
1  2185-0

In [9]:
# load chartevents table and find the rows with HR_ITEM_IDs, get only 10 rows
df = pd.read_sql_query("SELECT * FROM chartevents WHERE itemid IN ({}) LIMIT 10".format(','.join(['?']*len(HR_ITEM_IDs))), conn, params=HR_ITEM_IDs)
print(df)